# Imports

In [14]:
import plotly
import numpy as np
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression

# Raw Data
Data is in the form of the markdown table found in my lab notebook.

In [2]:
data = """
|6.0|1.609, 1.607, 1.609|
|6.1|1.646, 1.647, 1.646|
|6.2|1.688, 1.686, 1.688|
|6.3|1.734, 1.733, 1.732|
|6.4|1.775, 1.777, 1.778|
|6.5|1.817, 1.817, 1.817|
|6.6|1.851, 1.850, 1.850|
|6.7|1.897, 1.897, 1.897|
|6.8|1.940, 1.940, 1.940|
|6.9|1.982, 1.981, 1.982|
|7.0|2.023, 2.023, 2.023|
|7.1|2.069, 2.069, 2.068|
|7.2|2.104, 2.103, 2.103|
|7.3|2.146, 2.146, 2.146|
|7.4|2.184, 2.186, 2.186|
|7.5|2.230, 2.229, 2.230|
|7.6|2.269, 2.270, 2.269|
|7.7|2.313, 2.312, 2.313|
|7.8|2.357, 2.356, 2.357|
|7.9|2.416, 2.415, 2.416|
|8.0|2.414, 2.424, 2.424|
|8.1|2.471, 2.470, 2.469|
|8.2|2.517, 2.517, 2.517|
|8.3|2.559, 2.559, 2.560|
|8.4|2.603, 2.603, 2.603|
"""

# Preprocessing
Convert the data string into numpy arrays.

In [3]:
# Battery voltage, ADC measured voltage
v_batt = []
v_adc = []

# Get voltages from the whole v_batt operating range
for line in data.split('\n'):
    if line == '': continue
    line = line.split('|')
    v_batt.append(float(line[1]))
    v_adc.append([float(x) for x in line[2].split(', ')])
    
# Convert voltage arrays to numpy arrays
v_batt = np.array(v_batt)
v_adc = np.array(v_adc)

# Analysis
We start with a computation of the standard deviation of the ADC voltage measurements.

In [4]:
# Compute mean standard deviation across each row
print(f'Mean Standard Deviation of ADC Measurements: {v_adc.std(axis=1).mean():.6f}V')

Mean Standard Deviation of ADC Measurements: 0.000624V


Now we examine the mapping between $V_{\text{ADC}}$ and $V_{\text{batt}}$.

In [24]:
# Aggregate ADC voltages
v_adc_agg = v_adc.mean(axis=1)

# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=v_batt, 
    y=v_adc_agg, 
    mode='lines+markers',
    name='ADC Voltage'
))
fig.add_trace(go.Scatter(
    x=v_batt, 
    y=v_batt/3.5, 
    mode='lines+markers',
    name='Ideal ADC Voltage'
))

# Show the figure
fig.show()

Looks like the ADC voltage is a bit off of the ideal voltage, both in terms of weight and bias, in ML terms. Hence,
we look at a linear regression fit to the ADC voltage to compare to that of the ideal.

In [25]:
reg = LinearRegression().fit(v_batt.reshape(-1, 1), v_adc_agg)
print(f'vadc = {reg.coef_} * vbatt + {reg.intercept_}')

vadc = [0.41408462] * vbatt + -0.8766892307692311


$$V_{\text{IdealADC}} = 0.286 V_{\text{batt}}$$
$$V_{\text{ADC}} \approx 0.414 V_{\text{batt}} - 0.877$$

In [26]:
# Aggregate ADC voltages
v_adc_agg = v_adc.mean(axis=1)

# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=v_batt, 
    y=v_adc_agg, 
    mode='lines+markers',
    name='ADC Voltage'
))
fig.add_trace(go.Scatter(
    x=v_batt, 
    y=v_batt/3.5, 
    mode='lines+markers',
    name='Ideal ADC Voltage'
))
fig.add_trace(go.Scatter(
    x=[8.116], 
    y=[2.481], 
    mode='markers',
    name='ADC Voltage w/Battery'
))

# Show the figure
fig.show()